# Pound Drever Hall

## 0.0 Introduction

Pound–Drever–Hall (PDH) locking is a technique for extracting frequency offset information from a laser relative to a stable reference (e.g., a cavity). The method works by phase-modulating the laser beam to generate sidebands, which can be described using the Bessel expansion of a modulated sinusoid. The cavity reflects the carrier and sidebands differently, encoding frequency detuning into a phase shift of the reflected light. By demodulating this signal at the modulation frequency, we recover an error signal that is directly sensitive to frequency offset.

This notebook provides a visual introduction to these ideas before diving into the underlying math.

## 1.1 Sinusoid Modulation

The first thing we will do is create a time-domain sinusoid, this will represent our laser beam. We then modulate this sinusoid via an electro-optic modulator (EOM). The EOM will periodically phase-shift the signal at some given modulation frequency. Both the modulated and unmodulated carrier waves are visible below. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from scipy.special import j0, j1

# Generate time axis vectors
t1 = np.linspace(0, 1, 1000)
t2 = np.linspace(0, 10, 10000)

C = 3e8

freq = FloatSlider(value=4.4, min=0.1, max=10.0, step=0.1)
mod_freq = FloatSlider(value=0.9, min=0.1, max=10.0, step=0.1)
beta = FloatSlider(value=0.66, min=0.01, max=5.0, step=0.01)
phase_shift = FloatSlider(value=180, min=0, max=360, step=1)

refractive_index = FloatSlider(value=1.0, min=0.95, max=1.05, step=0.01)  # basically air
cav_freq = FloatSlider(value=15e9, min=13e9, max=17e9, step=10e6)  # around one FSR
amplitude_reflectivity = FloatSlider(value=0.95, min=0.8, max=1.0, step=0.01)


def sinusoid(freq, t, phase_shift_degrees=0):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t + np.pi * phase_shift_degrees / 180) 
    return y

def modulated_sinusoid(freq, mod_freq, beta, t):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t + beta * np.cos(2 * np.pi * mod_freq * t))
    return y

def get_sinusoid_spectrum(signal, t):
    N = len(t)
    dt = t[1] - t[0]
    Y = np.fft.fftshift(np.fft.fft(signal))
    f = np.fft.fftshift(np.fft.fftfreq(N, d=dt))
    Y_mag = np.abs(Y) / np.max(np.abs(Y))
    return f, Y_mag

def reflected_sinusoid(freq, mod_freq, beta, phase_shift, t):
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=t) + sinusoid(freq=freq, phase_shift_degrees=phase_shift, t=t)
    return y

def beat_signal(freq, mod_freq, beta, phase_shift, t):
    y1 = reflected_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t)
    y2 = sinusoid(freq=mod_freq, t=t)
    return y1 * y2

def beat_signal_vs_phase(freq, mod_freq, beta, t, n_points=361):
    phases = np.linspace(0, 360, n_points)
    means = []
    for phi in phases:
        y = beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phi, t=t)
        means.append(np.mean(y))
    return phases, np.array(means)

def beat_signal_vs_phase_normalized(freq, mod_freq, beta, t, n_points=361):
    phases = np.linspace(0, 360, n_points)
    means = []
    y_norm = np.mean(beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=180, t=t))
    for phi in phases:
        y = beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phi, t=t) - y_norm
        means.append(np.mean(y))
    return phases, np.array(means)


def ref_coefficient(frequency, cav_len, r, n):
    omega = frequency * 2 * np.pi
    fsr = C / (2*n*cav_len)
    z = np.exp(1j * omega / fsr)
    return r * (z - 1)/(1 - r**2 * z)


def pdh_error(omega, wm, r, n, cav_len, demod_offset):
    theta = demod_offset * np.pi / 180
    
    F0 = ref_coefficient(frequency=omega, cav_len=cav_len, r=r, n=n)                  
    Fp = ref_coefficient(frequency=omega + wm, cav_len=cav_len, r=r, n=n)              
    Fm = ref_coefficient(frequency=omega - wm, cav_len=cav_len, r=r, n=n)           
    
    term = F0 * np.conj(Fm) - np.conj(F0) * Fp
    
    error = np.cos(theta) * np.imag(term) + np.sin(theta) * np.real(term)
    return error



def reflected_power_pdh(omega, wm, beta, r, n, cav_len):
    F0 = ref_coefficient(frequency=omega, cav_len=cav_len, r=r, n=n)
    Fp = ref_coefficient(frequency=omega + wm, cav_len=cav_len, r=r, n=n)
    Fm = ref_coefficient(frequency=omega - wm, cav_len=cav_len, r=r, n=n)
    
    J0 = j0(beta)  
    J1 = j1(beta)  
    
    power = (J0**2 * np.abs(F0)**2 + 
             J1**2 * (np.abs(Fp)**2 + np.abs(Fm)**2))
    return power


In [2]:
def plot_sinusoid(freq):
    T = t1
    y = sinusoid(freq=freq, t=T)
    plt.figure(figsize=(8,4))
    plt.plot(T, y, label=fr'cos(2pi * {freq}t)')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Unmodulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_modulated_sinusoid(freq, mod_freq, beta):
    T = t1
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=T)
    plt.figure(figsize=(8,4))
    plt.plot(T, y, label=fr'cos(2pi * {freq} * t + {beta:.2e}*cos(2pi*{mod_freq:.2e}t))')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Modulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_sinusoid, freq=freq)
interact(plot_modulated_sinusoid, freq=freq, mod_freq=mod_freq, beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), Output()), _dom_classes=(…

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_modulated_sinusoid(freq, mod_freq, beta)>

If we look at the modulated signal we see something that seems similar to multiple sinusoidal signals with different frequencies superimposing themselves on each other. This can be proven with the Jacobi-Anger identity but for now we'll just grab the frequency spectrum and see what happens. 

In [3]:

def plot_spectrum_modulated_sinusoid(freq, mod_freq, beta):
    T = t2
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=T)
    f, Y_mag = get_sinusoid_spectrum(signal=y, t=T)
    plt.figure(figsize=(10,5))
    plt.plot(f, Y_mag)
    plt.title("Frequency Spectrum of Phase-Modulated Signal")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Normalized Magnitude")
    plt.xlim(freq-5*mod_freq, freq+5*mod_freq)
    plt.grid(True)
    plt.show()

interact(plot_spectrum_modulated_sinusoid, 
         freq=freq, 
         mod_freq=mod_freq, 
         beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_spectrum_modulated_sinusoid(freq, mod_freq, beta)>

We can see that for modulation frequencies less than the carrier frequency, and for comparatively small values of beta, we generally observe two prominent 'sidebands' equidistant on each side of the dominant band (which as you'd expect is the carrier frequency). For now we can assume that a properly designed PDH system will ensure that the modulation frequency is much lower than the carrier frequency and that our beta is fairly small.

## 1.2 Cavity interference
We can make the assumption that our modulating signal is very resistant to frequency drift. If we could find a way to translate frequency offset into a phase offset, we could then get a beat signal by superimposing the two fields and demodulating them with respect to our modulating tone. Taking the DC term of this signal would get us a value that is bijective (within our operating range) with the phase offset and thus the frequency offset. In other words, if the beat signal is odd-symmetric around our modulating tone, demodulating it with respect to that tone will allow us to obtain the magnitude and sign of our frequency offset. In other words, we should:

1. Impart a phase-shift on a signal with respect to frequency drift that is odd-symmetric
2. Make the phase shift as sharp as possible so we can detect it easily
3. Center it on our target carrier frequency that we are trying to correct to

We accomplish the three objectives above through either a Fabry-Perot cavity or a ring resonator. For what we care about they are functionally the same. We rotate the carrier with respect to frequency offset and demodulate the reflected field with respect to the modulation frequency.

For the code below, we assume a perfect cavity respose i.e. only the carrier component of our incident field is reflected. This is done to simplify the code. 

In [4]:
def plot_reflected_sinusoid(freq, mod_freq, beta, phase_shift):
    y = reflected_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t1)
    yf = reflected_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t2)
    f, Y_mag = get_sinusoid_spectrum(signal=yf, t=t2)
    
    plt.figure(figsize=(8,4))
    plt.plot(t1, y)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Reflected Field")
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10,5))
    plt.plot(f, Y_mag)
    plt.title("Frequency Spectrum of Reflected Field")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Normalized Magnitude")
    plt.xlim(freq-5*mod_freq, freq+5*mod_freq)
    plt.grid(True)
    plt.show()

interact(plot_reflected_sinusoid, freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.90000…

<function __main__.plot_reflected_sinusoid(freq, mod_freq, beta, phase_shift)>

## 1.3 Beat Signal

We can see that the time-domain representation of our reflected field takes on an interesting shape, and looking at the frequency-domain representation of that field we can see that for small phase offsets around a 180-degree baseline that the carrier tone is mostly eliminated (there are a couple reasons it doesn't disappear completely but that's not relevant for now). Now we can demodulate this reflection to obtain our beat signal. 

In [5]:
def plot_beat_signal(freq, mod_freq, beta, phase_shift):
    y = beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t1)

    dc_term = np.mean(y)
    print(f"Mean (DC component) of beat signal: {dc_term:.6f}")
    
    plt.figure(figsize=(8,4))
    plt.plot(t1, y, label="Beat signal")
    plt.axhline(dc_term, color="red", linestyle="--", label=f"Mean = {dc_term:.3e}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Beat Signal")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_beat_signal, freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.90000…

<function __main__.plot_beat_signal(freq, mod_freq, beta, phase_shift)>

We can see from the above that if our modulated signal is properly formed (two prominent sidebands), then small phase adjustments around the baseline rotation of 180 degrees results in a mean value that appears to be odd-symmetric and increasing/decreasing monotonically with respect to phase shift from baseline. The graph below plots this relationship out explicitly.  

In [6]:
def plot_beat_signal_vs_phase(freq, mod_freq, beta):
    phis, ys = beat_signal_vs_phase(freq=freq, mod_freq=mod_freq, beta=beta, t=t1)

    idx = np.argmin(np.abs(phis - 180))
    intercept_phase = phis[idx]
    intercept_value = ys[idx]

    plt.figure(figsize=(8,4))
    plt.plot(phis, ys, label="Mean beat signal")
    plt.axhline(0, color="black", linestyle="--", linewidth=0.8)

    plt.plot(intercept_phase, intercept_value, 'ro', label=f"180° intercept ({intercept_value:.3e})")
    plt.axvline(intercept_phase, color="red", linestyle="--", linewidth=0.8)

    plt.xlabel("Phase (Degrees)")
    plt.ylabel("DC Value")
    plt.title("Cavity Carrier Rotation vs Beat Signal DC Term")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_beat_signal_vs_phase, freq=freq, mod_freq=mod_freq, beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.90000…

<function __main__.plot_beat_signal_vs_phase(freq, mod_freq, beta)>

What we are looking at above can be thought of as a sort of toy representation of a PDH error signal. The basic shape is similar but a lot of factors present in a real-world implementation were not accounted for. We also haven't bothered to compute phase offset from frequency offset and display the error signal as a result of a frequency sweep, which would probably be more useful. Rather, we just assume that the cavity does its job properly and rotates our carrier term by some given amount depending on its frequency offset.

However, we can still see that this 'Toy Error Signal' is odd-symmetric around our 180-degree offset and is also monotonic in either direction up to a given point. Past this point the derivative changes signs and our error signal is no longer reliable (the bijection from frequency offset to beat signal DC term alluded to earlier breaks down). It's natural to ask if one should make sure to avoid excessive phase offsets (and by extension excessive frequency offsets). This is why servo controllers are combined with a separate acquisition strategy (e.g. sweep-and-hold, or an auxiliary wide-range actuator) to get the laser close enough that the PDH slope is reliable. 

We also see that there is generally some small offset at 180 degrees (cavity resonance). If we know what this offset, which can be obtained from the cavity response transfer function, we can normalize accordingly. 

In [7]:
def plot_beat_signal_vs_phase_normalized(freq, mod_freq, beta):
    phis, ys = beat_signal_vs_phase_normalized(freq=freq, mod_freq=mod_freq, beta=beta, t=t1)
    idx = np.argmin(np.abs(phis - 180))
    intercept_phase = phis[idx]
    intercept_value = ys[idx]
    plt.figure(figsize=(8,4))
    plt.plot(phis, ys, label="Mean beat signal")
    plt.axhline(0, color="black", linestyle="--", linewidth=0.8)
    plt.plot(intercept_phase, intercept_value, 'ro', label=f"180° intercept ({intercept_value:.3e})")
    plt.axvline(intercept_phase, color="red", linestyle="--", linewidth=0.8)
    plt.xlabel("Phase (Degrees)")
    plt.ylabel("DC Value")
    plt.title("Cavity Carrier Rotation vs Normalized Beat Signal DC Term")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_beat_signal_vs_phase_normalized, freq=freq, mod_freq=mod_freq, beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.90000…

<function __main__.plot_beat_signal_vs_phase_normalized(freq, mod_freq, beta)>

## 2.1 Bessel Expansion

As mentioned earlier, PDH exploits the frequency domain characteristics of a modulated sinusoid, these characteristics are visible in the Bessel function expansion of the signal via the Jacobi-Anger identity. 


$$
e^{i\beta \sin(\omega_mt)} = \sum_{n=-\infty}^\infty J_n(\beta)e^{in\omega_mt}
$$

Given our modulated sinusoid signal, as visible from the code:

$$
cos(\omega t + \beta \sin(\omega_mt)) = \mathbb{R}\{e^{i\omega t}e^{i\beta \sin(\omega_m t)}\} = \mathbb{R}\{e^{i\omega t}\sum_{n=-\infty}^\infty J_n(\beta)e^{in\omega_m t}\}
$$

Where

$$
J_n(\beta) = \frac{1}{2\pi}\int_0^{2\pi}\cos(n\theta - \beta \sin(\theta))d\theta
$$

It's relatively easy to see from the integral above that for small values of beta and large values of n, the overall magnitude will be quite low. Integrating a high-frequency sinusoid across a large number of cycles will tend to yield a very small average value. The beta scaled sinusoidal offset can counter this by introducing a DC term into beat signal when we demodulate the the series but for small values of beta this DC term is negligible. In other words, we can say that for small values of beta we only really care about the first two terms $J_0$ and $J_1$

Evaluating 1. we get:

$$
cos(\omega t + \beta \sin(\omega_m t)) \approx J_0(\beta)\cos(\omega t) - J_1(\beta)[\cos((\omega + \omega_m)t) - \cos((\omega - \omega_m)t)]
$$

Where:
$$
J_0(\beta) \approx 1 - \frac{\beta^2}{4} \newline J_1(\beta) \approx \frac{\beta}{2}
$$

More precisely:

#### Equation 1: Bessel Expansion of Modulated Sinusoid
$$
E_{ref} \approx E_0 * (F(\omega)J_0(\beta)e^{i \omega t} - J_1(\beta) * [F(\omega + \omega_m)e^{i(\omega + \omega_m)t} - F(\omega - \omega_m)e^{i(\omega - \omega_m)t}])
$$


What all this means is that if our beta is small, when we take the frequency spectrum of our signal we should see three main contributions present. One large one at the carrier frequency and two smaller ones at the carrier frequency plus or minus the modulation frequency. These are the sidebands. This is easily verifiable by playing around with beta in the frequency spectrum plots above. When beta is made larger the other Bessel functions are able to contribute more and the frequency spectrum acquires more spikes. We don't want this so we make sure beta is small.


## 2.2.1 Fabry Perot Cavity
We have our current toy PDH error signal but it is not really reflective of what a real PDH error signal looks like beyond the general idea of being odd-symmetric around a central point. The next step is to get a realistic signal.

We define our fields as follows:

$$
E_{inc} = E_0e^{i\omega t}\newline
E_{ref} = E_1e^{i\omega t}
$$
Where $E_0$ and $E_1$ are complex constants in order to account for phase differences between the two signals.


Assuming a symmetric Fabry-Perot cavity with no losses, the reflection coefficient can be obtained as follows:

#### Equation 2: Reflection Coefficient

$$
F(\omega) = \frac{E_{ref}}{E_{inc}} = \frac{r(e^{i\frac{\omega}{\Delta v_{fsr}}}-1)}{1-r^2e^{i\frac{\omega}{\Delta v_{fsr}}}}
$$

Where $r$ is the amplitude reflection coefficient and $\Delta v_{fsr} = \frac{c}{2L}$ is the free spectral range (distance between modes) for a cavity with length $L$.


We can think of the reflection coefficient as a complex-valued vector with a phase angle dependent on the carrier frequency. The reflected field is then simply the incident field stretched and rotated in the complex plane by this reflection coefficient vector. Most of the time the reflection coefficient vector is relatively real-valued and constant magnitude. However, when approaching resonance it becomes almost entirely imaginary and flips over the real axis from +-i to -+i during resonance. At resonance the magnitude is zero. This sign flip also means that the cavity response is odd-symmetric about resonance. Below is a visualization:

In [8]:
SPAN = 1.2
N = 4001

def plot_ref_coefficient(frequency, r, n):
    cav_len = 0.01
    F = ref_coefficient(frequency=frequency, r=r, n=n, cav_len=cav_len)

    plt.figure(figsize=(6,6))
    plt.quiver(0, 0, F.real, F.imag,
               angles='xy', scale_units='xy', scale=1,
               color='b', label=f'F(f), |F|={abs(F):.2f}')

    plt.axhline(0, color='k', lw=0.5)
    plt.axvline(0, color='k', lw=0.5)
    plt.gca().set_aspect('equal', 'box')

    plt.xlim(-1.2, 1.2)
    plt.ylim(-1.2, 1.2)

    plt.title('Fabry-Pérot Reflection Coefficient F(f)')
    plt.xlabel('Re{F}')
    plt.ylabel('Im{F}')
    plt.legend()
    plt.grid(True)

    fsr = C / (2*n*cav_len)
    nu = np.linspace(-SPAN*fsr/2, SPAN*fsr/2, N)
    x = nu/fsr
    phi = (2 * np.pi * x)
    z = np.exp(1j * phi)
    K = (r * (z - 1)) / (1 - (r**2) * z)
    mag = np.abs(K)
    phase = np.unwrap(np.angle(K))  # radians

    plt.figure(figsize=(10,4))
    plt.plot(x, mag, lw=2)
    plt.xlabel('Detuning f / FSR')
    plt.ylabel('|F(f)|')
    plt.title('Reflected Amplitude |F(f)| vs Detuning')
    plt.grid(True)

    plt.figure(figsize=(10,4))
    plt.plot(x, phase, lw=2)
    plt.xlabel('Detuning f / FSR')
    plt.ylabel('arg F(f)')
    plt.title('Reflected Phase arg F(f) vs Detuning')
    plt.grid(True)

    plt.show()


interact(plot_ref_coefficient, frequency=cav_freq, r=amplitude_reflectivity, n=refractive_index)

interactive(children=(FloatSlider(value=15000000000.0, description='frequency', max=17000000000.0, min=1300000…

<function __main__.plot_ref_coefficient(frequency, r, n)>

As we detune the frequency, the reflection coefficient $F(\omega)$ traces a circle on the left side of the complex plane, consistent with the literature. If we set $r=1$, this behavior disappears. This is because the input mirror is a perfect reflector, no leakage field exists, and the total reflection is a constant $\pi$-phase flip $F(\omega) = -1$ (purely real, no frequency dependence).  

For $r < 1$ (symmetric, lossless cavity), the interference between the prompt and leakage fields drives $|F|$ to zero at resonance, and $\arg(F)$ flips sign across resonance. This is exactly the odd-symmetric response PDH exploits.

For all other frequencies there is little transformation beyond the $\pi$-phase flip. This explains why we can expect our sidebands to not be strongly affected by a properly designed cavity.



## 2.2.2 Ring Resonator


A ring resonator is a roundabout for light that accomplishes much the same thing as an FP cavity but with a generally higher quality factor.

At resonance
$$
L = m \frac{\lambda}{n_{eff}}
$$

Where m is an integer.

From Maxwell's curl equations:

$$
\nabla \times (\nabla \times E) = -\mu_0\epsilon(r) \frac{\partial^2 E}{\partial t^2}
$$

All this is saying is that if we froze time and traced how the field curves through space, the rate at which that curvature changes equals some constant times how the field would accelerate in time if we let it evolve again. Anything that satisfies this equation is called a wave. 

Using the vector identity $\nabla \times \nabla \times E = \nabla(\nabla \cdot E) - \nabla^2E $

And assuming our dielectric substrate has no free charge in our particular location i.e. $\nabla \cdot [\epsilon(r)E] = 0$, or in other word the field is divergence free because it's not pumping change into or out of the dielectric, we get the vector Helmholtz equation:

$$
\nabla^2E + k_0^2n^2(r)E = 0
$$

with $k_0 = \omega/c$.

We now have a simplified expression for answering the question "what is a wave?". A waveguide, by the honor of its name, must guide a wave. Therefore, the material properties of the waveguide must let the wave wave. We are given these properties by whoever made the waveguide. We then solve the equation for "what is a wave?" given these properties as constraints.

A ring resonator has a ring waveguide and a through waveguide. Some of the incoming field goes in from the straight waveguide, loops around, and interferes with the field that went straight through. This means that we have two fields: a 'through' field that goes straight through, and a 'coupled' field that gets looped around. Both of these fields are waves, so they are valid answers to the question "what is a wave?"

$$
\nabla_t^2E_1 + [k_0^2n_1^2(x, y) - \beta_1^2]E_1 = 0
$$

$$
\nabla_t^2E_2 + [k_0^2n_2^2(x, y) - \beta_2^2]E_2 = 0
$$

Where beta just relates the effective refractive index $n_{eff}$ to the free space wavenumber $k_0$.

In the coupler region, the total field is approximately a superposition of the two isolated modes:
$$
E(x, y, z) = A_1(z)E_1(x, y)e^{j\beta_1 z} + A_2(z)E_2(x, y)e^{j\beta_2 z}
$$

This means total field is a standing that varies along the z axis assuming we treat x and y as constant. Substituting back into the Helmoholtz equation above (after using orthogonality and ignoring second derivative terms we get):

$$
\frac{dA_1}{dz} = -j\nabla\beta_{12}A_1 - jk_{12}A_2
$$
$$
\frac{dA_2}{dz} = -j\nabla\beta_{21}A_2 - jk_{21}A_1
$$

Then intuitively:

$$
k = \frac{\omega}{4P_1}\int\int\Delta\epsilon(x, y)E_1^*(x, y)\cdot E_2(x, y)dxdy
$$

Where $\Delta\epsilon = \epsilon(x, y) - \epsilon_1(x, y)$ and $P_1$ is the power normalization of mode 1. All we are doing here is defining a coupling coefficient at location z lying on the x-y plane (cross-sectional area of our waveguides). We correlate the dot product of the two fields weighted by the electromagnetic permitivity difference at that particular point on the plane to determine the coupling coeffcient at that particular location z. We assume weak coupling and reciprocal media i.e. $k_{12} = k_{21} = k$.

Let $A_i(z) = a_i(z)e^{j\beta z} with \beta\approx\beta_1\approx\beta_2$. This provides a generic answer to the "what is a wave" question. Then:

$$
\frac{da_1(z)}{dz} = jka_2, \frac{da_2}{dz} = jka_1
$$

Assuming the guides are phase-matched. Differentiating these terms once more gives us a solution to our "what is a wave" question:

$$
\frac{d^2a_1(z)}{dz^2} \rightarrow a_1(z) = a_1(0)cos(kz) + ja_2(0)sin(kz)
$$
$$
a_2(z) = a_2(0)cos(kz) + ja_1(0)sin(kz)
$$

We can represent this with a matrix:
$$
\begin{bmatrix}
a_1(z) \\
a_2(z)
\end{bmatrix}
=
\begin{bmatrix}
\cos(kz) & j\sin(kz) \\
j\sin(kz) & \cos(kz)
\end{bmatrix}
\begin{bmatrix}
a_1(0) \\
a_2(0)
\end{bmatrix}$$

$a_1(0)$ and $a_2(0)$ are the oscillating fields at coupling location $z=0$ on waveguides 1 and 2 respectively. The superposition of these oscillating fields forms a stationary wave (supermode). The matrix is a way to answer the question "based on the definition of what a wave is and the material properties of our system, if we were to stop time and walk along the z direction, what would be our amplitude and phase after some number of steps?". This matrix can then be used to also find the magnitude and phase at any point along our supermode.

Since the wavelength changes with frequency, the superposition of these two waves will change when the phase alignment of the through and the coupled wave changes. 

This means that we can encode frequency offset into the phase/amplitude of our supermode. We now have everything we need to define our transfer function. We could do the math explicitly but it's just greasy algebra and we already covered the fun stuff. The one thing to remember is each field also gets coupled on the way out of the loop meaning some fraction still remains afterwards. This creates a convergent series. Math can be found [here](../Resources/rings.pdf):

$$
F(\omega) = \frac{E_{out}}{E_{in}} = \frac{\gamma - \alpha e^{-j\beta L}}{1 - \gamma\alpha e^{-j\beta L}}
$$

Where $L = m\frac{\lambda}{n_{eff}}$, $\lambda = 2\pi\frac{c}{w}$, and $\gamma = cos(kz)$ where $z=L_c$.

## 2.3 Power Signal

Define incident field:

$$
E_{inc} = E_0e^{i\omega t + \beta \cos(\omega_m t)}
$$

From 2.1:
$$
= E_0 * (J_0(\beta)\cos(\omega t) - J_1(\beta)[\cos((\omega + \omega_m)t) - \cos((\omega - \omega_m)t)])
$$

Where:
$$
J_0(\beta) \approx 1 - \frac{\beta^2}{4} \newline J_1(\beta) \approx \frac{\beta}{2}
$$


Then, given our reflection coefficient $F(\omega)$:

$$
E_{ref} = F(\omega)E_{inc} = E_0 * (F(\omega)J_0(\beta)\cos(\omega t) - J_1(\beta) * [F(\omega + \omega_m)\cos((\omega + \omega_m)t) - F(\omega - \omega_m)\cos((\omega - \omega_m)t)])
$$


We can also represent it in its complex exponential form:

$$
E_{ref} = F(\omega)E_{inc} = E_0 * (F(\omega)J_0(\beta)e^{i \omega t} - J_1(\beta) * [F(\omega + \omega_m)e^{i(\omega + \omega_m)t} - F(\omega - \omega_m)e^{i(\omega - \omega_m)t}])
$$

Where $P_{ref} = |E_{ref}|^2$:


$$
P_{\rm ref}
= P_c\,|F(\omega)|^2
+ P_s\big(|F(\omega+\omega_m)|^2 + |F(\omega-\omega_m)|^2\big) \newline
+ 2\sqrt{P_c P_s}\,\Big\{
\Re\!\big[F(\omega)F^*(\omega+\omega_m) - F^*(\omega)F(\omega-\omega_m)\big]\cos(\omega_m t)\newline
+ \Im\!\big[F(\omega)F^*(\omega+\omega_m) - F^*(\omega)F(\omega-\omega_m)\big]\sin(\omega_m t)
\Big\}\newline
+ \text{(2$\omega_m$ terms)}\newline
$$

Since we don't care about constants or higher-order harmonics as they will demodulate to zero with respect to the base harmonic, we get:

#### Equation 3: Power Signal
$$
P_{\rm ref} \approx \Re\{(F(\omega)F^*(\omega+\omega_m) - F^*(\omega)F(\omega-\omega_m))e^{i\omega_m t}\}
$$

Where:

$$
P_c = J_0^2(\beta)P_0
$$

$$
P_s = J_1^2(\beta)P_0
$$

$$
P_0 \approx P_c + 2P_s
$$

In [9]:
def plot_resonance(wm, beta, r, n, carrier_freq, cav_len, offset):
    omega_carrier = 2 * np.pi * carrier_freq
    detunings = np.linspace(0.9, 1.1, 1000)
    frequencies = omega_carrier * detunings
    
    P = [reflected_power_pdh(w, wm, beta, r, n, cav_len) for w in frequencies]
    err = [pdh_error(w, wm, r, n, cav_len, offset) for w in frequencies]

    P_normalized = P / np.max(np.abs(P))
    err_normalized = err / np.max(np.abs(err))

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    ax1.plot(detunings, P_normalized, label="Reflected Power", color="blue", linewidth=2)
    ax1.set_ylabel("Normalized Reflected Power", color="blue")
    ax1.tick_params(axis='y', labelcolor="blue")
    ax1.set_ylim(-0.1, 1.1)
    ax1.grid(True, alpha=0.3)
    ax1.set_title(f"Reflected Power (Modulation: {wm:.2e} Hz\nCarrier: {carrier_freq:.2e} Hz)")
    ax1.axvline(1.0, color='red', linestyle='--', alpha=0.7, linewidth=1)
    
    ax2.plot(detunings, err_normalized, label="PDH Error", color="red", linewidth=2)
    ax2.set_xlabel("ω / ω_res")
    ax2.set_ylabel("Normalized Error Signal", color="red")
    ax2.tick_params(axis='y', labelcolor="red")
    ax2.set_ylim(-1.1, 1.1)
    ax2.grid(True, alpha=0.3)
    ax2.set_title("PDH Error Signal")
    ax2.axhline(0, color='black', linestyle='-', alpha=0.5)
    ax2.axvline(1.0, color='red', linestyle='--', alpha=0.7, linewidth=1)

    plt.tight_layout()
    plt.show()


carrier_slider = FloatSlider(value=100e16, min=10e6, max=100e12, step=1e6, 
                       description="Carrier")
cav_slider = FloatSlider(value=0.1e-5, min=0.01e-5, max=0.4e-5, step=1e-8, 
                       description="Cavity Length")
wm_slider = FloatSlider(value=10e6, min=10e6, max=100e12, step=1e6, 
                       description="Mod")
r_slider = FloatSlider(value=0.96, min=0.9, max=1.0, step=0.01, 
                       description="Reflection Coeff")
beta_slider = FloatSlider(value=0.3, min=0.1, max=1.4, step=0.04, 
                       description="Beta")
n_slider = FloatSlider(value=1.0, min=0.9, max=1.1, step=0.001, 
                       description="Refraction Coeff")
offset_slider = FloatSlider(value=0, min=-180, max=180, step=1, 
                       description="Modulation offset")

interact(plot_resonance, carrier_freq=carrier_slider, wm=wm_slider, beta=beta_slider, r=r_slider, n=n_slider, cav_len=cav_slider, offset=offset_slider)



interactive(children=(FloatSlider(value=10000000.0, description='Mod', max=100000000000000.0, min=10000000.0, …

<function __main__.plot_resonance(wm, beta, r, n, carrier_freq, cav_len, offset)>

From the above, we can make some observations:

1. Smaller beta values lead to a deeper resonance trough. This may explain why it's referred to as modulation depth. Physically it's proportional to the modulation voltage amplitude. 

2. Reflection amplitude coefficient approaching 1 leads to sharper resonance features coherent with what was observed in the $F(\omega)$ plot above. As expected, everything breaks at $r=1$ because there is no leakage beam.

3. Changing the refractive index or the cavity length shifts both plots across the x axis, this makes sense as the resonance frequency would also be changing at this point (and would be visible as some detuning number times the original carrier freq).

4. Changing the carrier while keeping the above fixed would do much of the same thing for much of the same reasons.

5. Changing the modulation frequency so that it's relatively close to the carrier does some weird things to the error signal's shape. 

6. Phase shifting the modulation signal appears to rotate the error signal about the x-axis rather than moving it along the axis as one might naively expect. In other words, changing the phase of the complex function along any point of the detuning axis such that it's projection onto the real plane changes.


1 through 4 are boring, 5 and 6 seem cool. Below is a graph.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from matplotlib.lines import Line2D

def F_of_omega(omega, gamma=1.0):
    delta = omega
    return delta / (delta + 1j*gamma)

def delta_pdh(omega, wm, gamma=1.0):
    F0 = F_of_omega(omega, gamma)
    Fp = F_of_omega(omega + wm, gamma)
    Fm = F_of_omega(omega - wm, gamma)
    return F0*np.conj(Fm) - np.conj(F0)*Fp

def plot_rotating_L(detune=0.0, phase_shift=0.0, mod_freq=0.5):
    omega = detune * 4
    delta  = delta_pdh(omega, wm=mod_freq, gamma=1.0)     
    theta  = np.deg2rad(phase_shift)
    RTheta = np.exp(1j*theta)                               

    R0, I0 = np.real(delta), np.imag(delta)           
    real_leg_tip_baseline = R0 + 0j                 
    imag_leg_vec_baseline  = 0 + 1j*I0           

    real_leg_tip = RTheta * real_leg_tip_baseline
    imag_leg_tip = real_leg_tip + RTheta * imag_leg_vec_baseline
    tip_vec      = RTheta * (R0 + 1j*I0)           

    proj = tip_vec.real                            

    REAL_COLOR, IMAG_COLOR, RES_COLOR, PROJ_COLOR, AX_COLOR = "#1f77b4", "#d62728", "#2ca02c", "#9467bd", "#555"
    fig, ax = plt.subplots(figsize=(7,7))
    ax.set_aspect('equal', 'box')
    ax.spines[['top','right']].set_visible(False)
    ax.grid(True, alpha=0.15, linewidth=0.6)
    ax.set_xlabel('Real'); ax.set_ylabel('Imag')
    ax.set_title('PDH Phasor Graph (Rotating L)')

    mags = [np.abs(real_leg_tip), np.abs(imag_leg_tip), np.abs(tip_vec), 1.0]
    lim = 1.2*max(mags)
    ax.set_xlim(-lim, lim); ax.set_ylim(-lim, lim)
    ax.axhline(0, lw=1, color=AX_COLOR, alpha=0.6)
    ax.axvline(0, lw=1, color=AX_COLOR, alpha=0.6)

    ax.arrow(0, 0, real_leg_tip.real, real_leg_tip.imag,
             head_width=0.03*lim, head_length=0.05*lim, length_includes_head=True,
             color=REAL_COLOR, linewidth=2)


    ax.arrow(real_leg_tip.real, real_leg_tip.imag,
             (imag_leg_tip-real_leg_tip).real, (imag_leg_tip-real_leg_tip).imag,
             head_width=0.03*lim, head_length=0.05*lim, length_includes_head=True,
             color=IMAG_COLOR, linewidth=2)

    ax.arrow(0, 0, tip_vec.real, tip_vec.imag,
             head_width=0.03*lim, head_length=0.05*lim, length_includes_head=True,
             color=RES_COLOR, alpha=0.7, linewidth=2)

    ax.arrow(0, 0, proj, 0,
             head_width=0.03*lim, head_length=0.05*lim, length_includes_head=True,
             color=PROJ_COLOR, linestyle='--', alpha=0.9, linewidth=2)

    mag, ang = np.abs(tip_vec), np.angle(tip_vec, deg=True)


    proxies = [Line2D([0],[0], color=REAL_COLOR, lw=3),
               Line2D([0],[0], color=IMAG_COLOR, lw=3),
               Line2D([0],[0], color=RES_COLOR, lw=3, alpha=0.7),
               Line2D([0],[0], color=PROJ_COLOR, lw=3, ls='--', alpha=0.9)]
    ax.legend(proxies, ["Real leg", "Imag leg", "Resultant Δ·e^{iθ}", "Projection Re{Δ·e^{iθ}}"],
              loc="lower right", frameon=True)

    plt.show()

interact(
    plot_rotating_L,
    detune=FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.01, description='detune'),
    phase_shift=FloatSlider(value=0.0, min=-360.0, max=360.0, step=1.0, description='phase_shift'),
    mod_freq=FloatSlider(value=0.5, min=0.05, max=2.0, step=0.01, description='mod_freq'),
)


interactive(children=(FloatSlider(value=0.0, description='detune', max=1.0, min=-1.0, step=0.01), FloatSlider(…

<function __main__.plot_rotating_L(detune=0.0, phase_shift=0.0, mod_freq=0.5)>

## 2.4 Error Signal

Understanding the diagram above is predicated on understanding the following expression denoted above in Equation 3:

$$
P_{\rm ref} \approx \Re\{(F(\omega)F^*(\omega+\omega_m) - F^*(\omega)F(\omega-\omega_m))e^{i\omega_m t}\}
$$

This is not the full $P_{ref}$, we are ignoring the DC terms and the higher-order harmonics, but it's the part that will actually get modulated so it's what we care about. 

$F(\omega)$ and $F(\omega_m)$ are complex constants, they have real and imaginary components. We add these components tip to tail to get our resultant vector, this vector rotates around our origin at a rate of $\omega_m$ by the complex exponential $e^{i\omega_m t}$ At any given time, the reflected power we observe is equal to this vector's projection over the real axis. This is effectively observed as a cosine wave with an amplitude equal to that of the resultant vector oscillating at modulation frequency. We can then demodulate this tone via a phase mixer and pass it through an LPF to get the DC term. This DC term is our error signal and is functionally equivalent to:


### Equation 4: Error Signal
$$
\epsilon = \frac{1}{2}\Re\{(F(\omega)F^*(\omega+\omega_m) - F^*(\omega)F(\omega-\omega_m))e^{i\phi}\}
$$

Where $\phi$ is the mixer offset with respect to the original modulating signal. Now we are able to explain some of the more complicated behaviour observable in the error signal graphs.


1. What detuning does: As we would expect, for low modulation frequencies the sign of our projection (purple arrow) flips as we move the detune slider from one end to the other. This is the odd-symmetric behaviour we would nominally expect. For high modulation frequencies the purple arrow goes back and forth quickly as the detuning slider is pulled further. This is coherent with what what is visible from error signal plots with modulation frequencies close to resonance.

2. What modulation phase shift does: Modulation phase shift effectively induces a quadrature between our resultant vector (green arrow) and our modulation phasor. It's helpful to think of this green arrow as the expression of a structure composed of two basis terms. Detuning from resonance and changing the modulation frequency will alter the shape of this structure, imparting a phase shift relative to the modulating tone will rotate it with respect to wherever the modulating phasor is. Both phasors rotate at the modulation frequency $\omega_m$. On the error signal graph this is also visible. If you look closely you will see that shifting the modulation phase rotates the error signal about the x axis.


We see that imparting a phase shift on our modulator rotates the green vector, which represents the complex coefficient $\gamma = F(\omega)F^*(\omega+\omega_m) - F^*(\omega)F(\omega-\omega_m)$. This vector is sensitive to detuning and our error signal is it's projection over the real axis. We want to make it such that the purple arrow moves as rapidly as possible outwards in either direction when we detune from resonance. The way we do this is by aligning our gamma such that it lies directly along the real axis, and thus in-phase with our modulating tone, as much as possible throughout our detuning range. Doing so will give us the largest possible dot product between the derivative of gamma with respect to frequency and the modulation vector. On the error signal graph this corresponds to the modulation phase shift that produces the sharpest possible slope about resonance.


## Further Reading
The above notebook made use of concepts and topics from [this paper](../Resources/BlackAJP01.pdf). More detailed math on how Bessel functions work can be found [here](../Resources/bessel.pdf).

